In [0]:
import tensorflow as tf

In [0]:
class MobileNet_classification(tf.keras.layers.Layer):
  def __init__(self, pooling='avg', classes=1):
    super(MobileNet_classification, self).__init__()
    self.classes = classes
    self.pooling = pooling

    self.avg_pooling = tf.keras.layers.GlobalAveragePooling2D()
    self.max_pooling = tf.keras.layers.GlobalMaxPooling2D()

    self.sigmoid_fc = tf.keras.layers.Dense(units=classes, activation=tf.keras.activations.sigmoid)
    self.softmax_fc = tf.keras.layers.Dense(units=classes, activation=tf.keras.activations.softmax)


  def call(self, inputs):
    if self.pooling == 'avg':
      x = self.avg_pooling(inputs)
    elif self.pooling == 'max':
      x = self.max_pooling(inputs)

    if self.classes == 1:
      x = self.sigmoid_fc(x)
    else:
      x = self.softmax_fc(x)
      
    return x

class depthwise_separable_convolution(tf.keras.layers.Layer):
  def __init__(self, filter_num, stride, padd="same",use_bias=False):
    super(depthwise_separable_convolution, self).__init__()
    #dw
    self.dw_conv_1 = tf.keras.layers.DepthwiseConv2D(kernel_size=(3, 3),
                                                strides=stride,
                                                padding=padd,
                                                use_bias=use_bias)
    self.bn1 = tf.keras.layers.BatchNormalization()

    #pw
    self.conv_2 = tf.keras.layers.Conv2D(filters=filter_num,
                                        kernel_size=(1, 1),
                                        strides=1,
                                        padding="same",
                                        use_bias=use_bias)
    self.bn2 = tf.keras.layers.BatchNormalization()

  def call(self, inputs):
    x = self.dw_conv_1(inputs)
    x = self.bn1(x)
    x = tf.nn.relu(x)

    x = self.conv_2(x)
    x = self.bn2(x)
    x = tf.nn.relu(x)
    
    return x
    

class MobileNet(tf.keras.Model):
  def __init__(self, include_top=True, classes=1000):
    super(MobileNet, self).__init__()

    self.include_top = include_top
    self.classes = classes

    self.zero_padd_1 = tf.keras.layers.ZeroPadding2D(padding=((0, 1), (0, 1)))
    self.conv_1 = tf.keras.layers.Conv2D(filters=32,
                                        kernel_size=(3, 3),
                                        strides=2,
                                        padding="valid",
                                        use_bias=False)
    self.bn_1 = tf.keras.layers.BatchNormalization()


    self.dw_separable_block_1 = depthwise_separable_convolution(filter_num=64,
                                                                stride=1,
                                                                padd="same")
    
    self.zero_padd_2 = tf.keras.layers.ZeroPadding2D(padding=((0, 1), (0, 1)))

    self.dw_separable_block_2 = depthwise_separable_convolution(filter_num=128,
                                                                stride=2,
                                                                padd="valid")
    
    self.dw_separable_block_3 = depthwise_separable_convolution(filter_num=128,
                                                                stride=1,
                                                                padd="same")
    
    self.zero_padd_3 = tf.keras.layers.ZeroPadding2D(padding=((0, 1), (0, 1)))
    
    self.dw_separable_block_4 = depthwise_separable_convolution(filter_num=256,
                                                                stride=2,
                                                                padd="valid")
    
    self.dw_separable_block_5 = depthwise_separable_convolution(filter_num=256,
                                                                stride=1,
                                                                padd="same")
    
    self.zero_padd_4 = tf.keras.layers.ZeroPadding2D(padding=((0, 1), (0, 1)))
    
    self.dw_separable_block_6 = depthwise_separable_convolution(filter_num=512,
                                                                stride=2,
                                                                padd="valid")
    
    self.dw_separable_block_7 = depthwise_separable_convolution(filter_num=512,
                                                                stride=1,
                                                                padd="same")
    
    self.zero_padd_5 = tf.keras.layers.ZeroPadding2D(padding=((0, 1), (0, 1)))
    
    self.dw_separable_block_8 = depthwise_separable_convolution(filter_num=1204,
                                                                stride=2,
                                                                padd="valid")
    
    self.dw_separable_block_9 = depthwise_separable_convolution(filter_num=1024,
                                                                stride=2,
                                                                padd="same")
    self.fc_1 = MobileNet_classification(pooling='avg', 
                                         classes=1)
  def call(self, inputs):
    x = self.zero_padd_1(inputs)
    x = self.conv_1(x)
    x = self.bn_1(x)
    x = tf.nn.relu(x)
    x = self.dw_separable_block_1(x)
    x = self.zero_padd_2(x)
    x = self.dw_separable_block_2(x)
    x = self.dw_separable_block_3(x)
    x = self.zero_padd_3(x)
    x = self.dw_separable_block_4(x)
    x = self.dw_separable_block_5(x)
    x = self.zero_padd_4(x)
    x = self.dw_separable_block_6(x)
    for _ in range(5):
      x = self.dw_separable_block_7(x)
    x = self.zero_padd_5(x)
    x = self.dw_separable_block_8(x)
    x = self.dw_separable_block_9(x)
    if self.include_top:
      x = self.fc_1(x)
    return x

In [0]:
def MobileNet_V1():
  return MobileNet(include_top=True, classes=1)